In [38]:
import pandas as pd
import numpy as np
import json

In [ ]:
df = pd.read_json("features/Kaisa_features.json")

df_expanded = pd.concat([
    df[col].apply(pd.Series).add_prefix(f'{col}_')
    for col in df.columns
], axis=1)

for i in range(10):
    items_col = f"{i}_items"
    items_expanded = df_expanded[items_col].apply(pd.Series)
    items_expanded.columns = [f"{items_col}_{j}" for j in range(items_expanded.shape[1])]
    df_expanded = pd.concat([df_expanded, items_expanded], axis=1)

#df_expanded.to_csv("features/Kaisa_features_expanded.csv", index=False)

In [31]:
df_expanded.describe().T

,count,mean,std,min,25%,50%,75%,max
0_championId,994.0,190.557344,261.762481,1.0,41.00,84.5,150.0,950.0
0_teamId,994.0,100.402414,6.334025,100.0,100.00,100.0,100.0,200.0
0_kills,994.0,5.838028,4.373881,0.0,3.00,5.0,8.0,28.0
0_deaths,994.0,6.019115,3.448753,0.0,4.00,6.0,8.0,20.0
0_assists,994.0,5.334004,4.206779,0.0,2.00,5.0,8.0,27.0
...,...,...,...,...,...,...,...,...
9_items_2,994.0,3254.252515,1580.688229,0.0,3010.25,3154.5,3869.0,8020.0
9_items_3,994.0,3248.474849,1708.891608,0.0,3009.00,3113.0,3869.0,8020.0
9_items_4,994.0,2659.671026,1838.705793,0.0,1031.00,3067.0,3165.0,8020.0
9_items_5,994.0,1756.681087,1672.854232,0.0,0.00,1057.0,3088.5,6701.0


In [32]:
df_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 994 entries, 0 to 993
Columns: 170 entries, 0_championId to 9_items_6
dtypes: bool(10), int64(140), object(20)
memory usage: 1.2+ MB


In [ ]:
lane_df = df_expanded[[f"{id}_lane" for id in range(10)]]
unique_in_col = lane_df.apply(lambda x: x.unique(), axis=0)
unique_in_col

0_lane                         [TOP]
1_lane                 [TOP, JUNGLE]
2_lane                 [JUNGLE, TOP]
3_lane              [JUNGLE, MIDDLE]
4_lane              [MIDDLE, JUNGLE]
5_lane      [MIDDLE, BOTTOM, JUNGLE]
6_lane              [BOTTOM, MIDDLE]
7_lane     [BOTTOM, UTILITY, MIDDLE]
8_lane             [UTILITY, BOTTOM]
9_lane    [UTILITY, Invalid, BOTTOM]
dtype: object

In [ ]:
df_same_lane = df_expanded[
    (df_expanded["0_lane"] == "TOP") & (df_expanded["1_lane"] == "TOP") &
    (df_expanded["2_lane"] == "JUNGLE") & (df_expanded["3_lane"] == "JUNGLE") &
    (df_expanded["4_lane"] == "MIDDLE") & (df_expanded["5_lane"] == "MIDDLE") &
    (df_expanded["6_lane"] == "BOTTOM") & (df_expanded["7_lane"] == "BOTTOM") & 
    (df_expanded["8_lane"] == "UTILITY") & (df_expanded["9_lane"] == "UTILITY")
]
lanes = [lane for lane in df_same_lane[]]

KeyError: 0

In [37]:
df_test = df_expanded[[col for col in df_expanded.columns if col.startswith("0")]]
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 994 entries, 0 to 993
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   0_championId   994 non-null    int64 
 1   0_teamId       994 non-null    int64 
 2   0_matchResult  994 non-null    bool  
 3   0_kills        994 non-null    int64 
 4   0_deaths       994 non-null    int64 
 5   0_assists      994 non-null    int64 
 6   0_goldEarned   994 non-null    int64 
 7   0_lane         994 non-null    object
 8   0_level        994 non-null    int64 
 9   0_items        994 non-null    object
 10  0_items_0      994 non-null    int64 
 11  0_items_1      994 non-null    int64 
 12  0_items_2      994 non-null    int64 
 13  0_items_3      994 non-null    int64 
 14  0_items_4      994 non-null    int64 
 15  0_items_5      994 non-null    int64 
 16  0_items_6      994 non-null    int64 
dtypes: bool(1), int64(14), object(2)
memory usage: 125.3+ KB


In [ ]:
# Load champion IDs
with open(f"champions.json", "r", encoding="utf-8") as f:
    json_data = json.load(f)
    champions_ids = [json_data[champion]["id"] for champion in json_data.keys()]

# Load Item IDs (Only legendary items)
with open(f"items/items.json", "r", encoding="utf-8") as f:
    json_data = json.load(f)
    item_ids = [json_data[str_id]["id"] for str_id in json_data.keys() if json_data[str_id]["tier"] in [3, 4]]
    item_ids = set(item_ids).intersection()

In [ ]:
113619